In [1]:
from bs4 import BeautifulSoup
import requests
from user_agent import generate_user_agent
import pandas as pd
import random
import time
from datetime import date
import collections
import numpy as np

Daily Mirror horoscope is on separate pages, and you can only access today, this week and this month. Only scraping for today here:

In [2]:
signs = ['Aries','Taurus','Gemini','Cancer','Leo','Virgo','Libra','Scorpio','Sagittarius','Capricorn','Aquarius','Pisces']
lst = []

for s in signs:
    
    #be a conscientious scraper and pause between scrapes and be tricky
    t = (random.randint(1,5)*0.1)
    time.sleep(t)
    page_link = 'https://www.mirror.co.uk/lifestyle/horoscopes/' + s.lower() + '/daily'
    #generate a user agent
    user_headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
    #get the page code
    page_response = requests.get(page_link, timeout=5, headers = user_headers)
    print (page_response)
    
    page_content = BeautifulSoup(page_response.content, "html.parser")
    reading = page_content.find('div', class_="star-text").get_text()
    
    lst.append([s, reading])
#make into DataFrame
mirrorhoro = pd.DataFrame(lst, columns = ['sign', 'reading'])
#save
filename = 'mirror_horo_' + str(date.today()) + '.csv'
mirrorhoro.to_csv(filename)

mirrorhoro

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


,sign,reading
0,Aries,"Hold back from offering advice, even when some..."
1,Taurus,You're ready to move forward with a big projec...
2,Gemini,You're confident about your ability to reach a...
3,Cancer,Use your imagination to come up with ways to l...
4,Leo,A personal project is making promising progres...
5,Virgo,You suspect your boss or a senior colleague mi...
6,Libra,A joint project appears to be going from bad t...
7,Scorpio,Be ready to accept a more active role in a joi...
8,Sagittarius,"If you are asked to carry out a specific job, ..."
9,Capricorn,If you hope to get the holiday or break you ar...


Daily Mail horoscope is all on one page. It only gives you today's horoscope, with links to get a personalised horoscope tucked away in the text.

In [3]:
signs = ['Aries','Taurus','Gemini','Cancer','Leo','Virgo','Libra','Scorpio','Sagittarius','Capricorn','Aquarius','Pisces']
page_link = "https://www.dailymail.co.uk/coffeebreak/horoscopes/index.html"
    
#generate a user agent
user_headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
#get the page code
page_response = requests.get(page_link, timeout=5, headers = user_headers)
print (page_response) #so I know it connected

#pick out the bits I need
page_content = BeautifulSoup(page_response.content, "html.parser")
section = page_content.find('div', id = "horoscope-forecast")
readings = section.find_all('dd')

sign = 0
lst = []
for x in range(1,(len(readings)),2): #not every dd is a reading
    reading = readings[x].get_text().replace("Now, please keep reading, give me your date of birth and I will tell you something wonderful about your future. Download your personal horoscope here.", "")
    lst.append([signs[sign], reading])
    sign += 1  #get next sign

mailhoro = pd.DataFrame(lst, columns = ['sign', 'reading'])
filename = 'mail_horo_' + str(date.today()) + '.csv'
mailhoro.to_csv(filename)

mailhoro

<Response [200]>


,sign,reading
0,Aries,"As the Sun moves into your sign, heralding the..."
1,Taurus,Smoking is such a peculiar habit. No one ever ...
2,Gemini,The believers of all major world religions hav...
3,Cancer,"It's as if you're hanging around, tapping your..."
4,Leo,How can you follow your heart if you're unsure...
5,Virgo,When we've learnt how to genetically modify hu...
6,Libra,Although on-screen action heroes have come to ...
7,Scorpio,Do you remember the early days of satnavs; bef...
8,Sagittarius,If you listen carefully you should be able to ...
9,Capricorn,The Supermoon Equinox is a clear signal that i...


The Sun has loads of historical horoscopes. All signs are on the same page. The daily horoscope is fitted around adverts and clickbait and all sorts. The code is written as a def, so that it can be used to harvest lists of pages.

In [4]:
def sun_scrape(str_page_address, str_save_yn):
    signs = ['Aries','Taurus','Gemini','Cancer','Leo','Virgo','Libra','Scorpio','Sagittarius','Capricorn','Aquarius','Pisces', 'No sign']
    horos = [[signs[x]] for x in range(0,12)]
    
    #generate a user agent
    user_headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
    
    #get the page
    page_response = requests.get(str_page_address, timeout=5, headers = user_headers)
    print (page_response)
    
    page_content = BeautifulSoup(page_response.content, "html.parser")
    div_article_content = page_content.find('div', class_="article__content")
    for div in div_article_content.find_all("div"): 
        div.decompose()    #get rid of all the divs from my chosen section
    ptags = div_article_content.find_all('p') #grab all the paras
    
    next_sign = 0
    current = 0    
    for x in range(len(ptags)):
        #check if tag is a sign heading
        if signs[current].upper() in ptags[x].get_text():
            #prevent capture of paras before the horoscopes - turn on capture with 1
            next_sign = 1        
            #prevent indexerror by limiting iteration of current
            if current < len(signs)-1:
                current += 1
            #add paras to identified sign, checking for next sign 
        elif next_sign == 1 and signs[current].upper() not in ptags[x].get_text():
            horos[current-1].append(ptags[x].get_text())  #not very elegant
    
    for x in range(len(horos)): #join together the paras into one cell - could probably be done in the previous loop
        reading = ' '.join(horos[x][2:])
        horos[x] = [horos[x][0], reading ]
    
    df= pd.DataFrame(horos, columns = ['sign', 'reading'])
    
    #save as a csv if requested
    if str_save_yn == 'y':
        filename = 'sun_horo_' + str(date.today()) + '.csv'
        df.to_csv(filename, encoding='utf-8-sig')
    
    return df


sunhoro = sun_scrape('https://www.thesun.co.uk/fabulous/horoscopes/8599923/mystic-meg-sunday-march-9/', 'y')

sunhoro

<Response [200]>


,sign,reading
0,Aries,This is a time for taking stock and growing in...
1,Taurus,This can be a truly outstanding week now that ...
2,Gemini,Your chart holds many delicious secrets right ...
3,Cancer,Even if to begin with you have a mixed view of...
4,Leo,Examine your ambitions closely and you may fin...
5,Virgo,You are so ready to learn on every level. Lett...
6,Libra,There is a strong strand\nof success in your c...
7,Scorpio,Relationships will go your way as emotional an...
8,Sagittarius,Jupiter and Mars inspire you to be your best b...
9,Capricorn,Your personal profile has a wild-at-heart comp...


Yahoo horoscopes are actually on the page, whereas other web providers have them on dynamic code and so they are harder to scrape. Also multiple outlets will use the same horoscope writer.
Yahoo puts its horoscopes on individual sign pages, and is pretty clean and easy. It has no historical ones.

In [5]:
signs = ['Aries','Taurus','Gemini','Cancer','Leo','Virgo','Libra','Scorpio','Sagittarius','Capricorn','Aquarius','Pisces']
lst = []

for sign in signs:
    
    #be a conscientious scraper and pause between scrapes
    t = (random.randint(1,21)*0.1)
    time.sleep(t)
    
    page_link = "https://www.yahoo.com/lifestyle/horoscope/"  + sign.lower() + "/daily-" + str(date.today()).replace('-','') + ".html"
    #generate a user agent
    user_headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
    #get the page code
    page_response = requests.get(page_link, timeout=5, headers = user_headers)
    print (page_response)
    
    page_content = BeautifulSoup(page_response.content, "html.parser")
    
    reading = page_content.find('div', class_= 'Fz(13px) Lh(1.9) Whs(n) C($c-fuji-grey-l) desc').get_text()
    reading = reading.replace(' Get Your Complete Cosmic Profile', '')
    lst.append([sign, reading])

yahoohoro= pd.DataFrame(lst, columns = ['sign', 'reading'])
filename = 'yahoo_horo_' + str(date.today()) + '.csv'
yahoohoro.to_csv(filename)

yahoohoro

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


,sign,reading
0,Aries,Youre proud and delighted when someones true c...
1,Taurus,If youve been toiling away on your own for too...
2,Gemini,A casual fling might have become far more mean...
3,Cancer,Youre doing your best to keep domestic matters...
4,Leo,You possess an innate knack for communicating ...
5,Virgo,Youre in the process of determining whats real...
6,Libra,The name of the game today is restoring balanc...
7,Scorpio,Keeping secrets is one of your signs superpowe...
8,Sagittarius,Spending time with a friendly gathering of lik...
9,Capricorn,An open-minded and reasonable approach leads t...


In [6]:
import string
def prep_and_count(df):
    
    #preparation for removing punctuation
    punct = '!"#$%&\()*+,-./:;<=>?@[\\]^_`{}~'
    transtab = str.maketrans(dict.fromkeys(punct, ''))
    #join all horos together, remove punctuation, make lower case
    v = ''.join(df.reading[0:11].tolist()).lower()
    w = v.translate(transtab).split()
    
    #count into a dictionary
    counts = collections.Counter(w)
    
    #create a dataframe from the count dictionary
    lst = []
    for x in counts.keys():
        lst.append([x, counts[x]])  #previously I needed .decode('utf-8') but somehow using translate earlier did this on its own
    counted_frame = pd.DataFrame(lst, columns = ['Word', 'Freq'])
    
    return counted_frame

In [7]:
def comparable(counts_frame):
    
    #take the frame of word counts, tweak and twiddle into something to compare with Eng_5000_words
    counts_frame.sort_values(by = 'Freq', ascending = False, inplace = True)
    counts_frame['RankHoro'] = [x for x in range(len(counts_frame))]
    counts_frame.reset_index(inplace = True)
    counts_frame.drop('index', axis = 1, inplace = True)
    
    return counts_frame

In [8]:
def eng_5000_base():
    eng_5000 = pd.read_csv('Eng_5000_words2.csv')
    
    #remove fussy parts-of-speech distinctions, sort and add new rankings
    #could just export a csv to refer to and remove this code
    tidy = eng_5000.groupby('Word').sum()
    tidy.sort_values('Frequency', ascending = False, inplace = True)
    tidy['Rank5000'] = [x for x in range(len(tidy))]
    
    return tidy

In [9]:
eng_5000 = eng_5000_base()

In [10]:
def compare_with_5000(comparable_df):    
    #merge horo word count with normal word count
    compare = pd.merge(comparable_df, eng_5000, right_on = 'Word', left_on = 'Word', how = 'inner')
    compare.drop(['Rank', 'Frequency', 'Freq'], axis = 1, inplace = True)

    #compare
    more_common = compare[compare.RankHoro < compare.Rank5000].copy()#needs .copy to show it's not a slice
    more_common['More'] = compare.Rank5000 - compare.RankHoro
    more_common.sort_values('More', ascending = False, inplace = True)
    return more_common

just_gimme_a_horo processes a csv and then goes through the above to produce a dataframe that compares the horo's word frequency with normal eng word frequency. It's not really that helpful - the sample size for the horo words is really small. You need to combine horos to get a bigger sample.

In [11]:
def just_gimme_a_horo(horo_csv):
    #access scraped horo
    df = pd.read_csv(horo_csv)
    df.drop([0], axis = 0, inplace = True) #until the scraper is sorted
    
    #prep and count
    df_counted =  prep_and_count(df)
    
    #prep and compare
    df_comp = comparable(df_counted)
    df_more_common = compare_with_5000(df_comp)
    
    return df_more_common

In [12]:
just_gimme_a_horo('mail_horo_2019-02-08.csv')

,Word,RankHoro,Rank5000,More
185,seldom,266,4335,4069
206,brave,298,4311,4013
274,instinct,421,3834,3413
203,ours,295,3707,3412
158,invisible,218,3399,3181
28,written,34,3154,3120
245,convinced,366,3390,3024
288,alien,447,3469,3022
166,subtle,232,3141,2909
148,confusion,204,3082,2878


### Try merging all the horos to get a bigger sample. In time we could do this with several days' worth of horos ###

In [13]:
#get the horos ready
def horos_to_merge(df_horo):
    
    #prep and count
    df_counted =  prep_and_count(df_horo)
    
    #prep and compare
    df_comp = comparable(df_counted)
    
    return df_comp

In [14]:
#get the horos from above and apply functions
dfmirror1 = horos_to_merge(mirrorhoro)
dfsun1 = horos_to_merge(sunhoro)
dfyahoo1 = horos_to_merge(yahoohoro)
dfmail1 = horos_to_merge(mailhoro)

#we just want their word counts
def rename (df, source):
    df.rename({'Freq' : 'Freq' + str(source)}, axis = 1, inplace = True)
    df.drop(['RankHoro'], axis=1, inplace = True)
    return df

#we would like to know which is which in case any stick out
dfmirror1 = rename(dfmirror1, 'mirror')
dfsun1 = rename(dfsun1, 'sun')
dfyahoo1 = rename(dfyahoo1, 'yahoo')
dfmail1 = rename(dfmail1, 'mail')

#merge all the horos
all_horos = pd.merge(dfmirror1, dfsun1, right_on = 'Word', left_on = 'Word', how = 'outer')\
            .merge(dfyahoo1, right_on = 'Word', left_on = 'Word', how = 'outer')\
            .merge(dfmail1, right_on = 'Word', left_on = 'Word', how = 'outer')

#remove nans, create column for total frequency of words in horos
all_horos.replace({np.nan:0}, inplace = True)
all_horos['Total freq'] = all_horos['Freqmirror'] + all_horos['Freqsun'] + all_horos['Freqyahoo'] + all_horos['Freqmail']

#add a ranking column, and tidy up
all_horos.sort_values(by = 'Total freq', ascending = False, inplace = True)
all_horos['RankAll'] = [x for x in range(len(all_horos))]
all_horos.reset_index(inplace = True)
all_horos.drop('index', axis = 1, inplace = True)

#save a copy as a csv
all_horos.to_csv('all_horos' + str(date.today()) + '.csv')

#make a comparison dataframe by merging eng5000 and combined horoscopes
compare = pd.merge(all_horos, eng_5000, right_on = 'Word', left_on = 'Word', how = 'inner')

#compare by ranking
df_more_common = compare[compare.RankAll < compare.Rank5000].copy()#needs .copy to show it's not a slice
df_more_common['More'] = compare.Rank5000 - compare.RankAll
df_more_common.sort_values('More', ascending = False, inplace = True)
df_more_common[['Word', 'Total freq','RankAll','Rank5000' ,'More']]

,Word,Total freq,RankAll,Rank5000,More
43,discount,11.0,50,3959,3909
321,charm,1.0,461,4265,3804
384,boring,1.0,598,4350,3752
309,harmony,1.0,432,4157,3725
193,known,2.0,235,3921,3686
218,fortunately,2.0,273,3879,3606
428,ribbon,1.0,696,4263,3567
200,invitation,2.0,247,3693,3446
453,emotionally,1.0,754,4196,3442
388,cease,1.0,614,4028,3414


That's quite distorted: words that only appear once in the horo are still much higher ranked than in the 5000 because there's nothing like 5000 words in the horo. 
Two ideas:
1. Maybe it would be better to compare the actual frequencies relative to their sample - ie **percent freq in horos against percent freq in eng 5000**
2. Maybe filter it so we only look at words that appear more than once or twice int he horos.

Let's do both together.

Bigger sample is almost certainly better.

In [15]:
#compare by percentage of total frequency
df_per_cent_common = df_more_common[df_more_common['Total freq'] > 2].copy()#get the words that appear more than twice
df_per_cent_common['Horos %'] = df_per_cent_common['Total freq'] / df_per_cent_common['Total freq'].sum()
df_per_cent_common['Eng 5000 %'] = df_per_cent_common['Frequency'] / df_per_cent_common['Frequency'].sum()
df_per_cent_common['More common'] = df_per_cent_common['Horos %'] - df_per_cent_common['Eng 5000 %']
df_per_cent_common.sort_values(by = 'More common',ascending = False, inplace = True)


top_sample = df_per_cent_common[df_per_cent_common['Horos %'] > df_per_cent_common['Eng 5000 %']]
top_sample[['Word', 'Total freq', 'Horos %', 'Frequency', 'Eng 5000 %', 'More common']]



,Word,Total freq,Horos %,Frequency,Eng 5000 %,More common
4,your,116.0,0.080221,659622.0,0.013839,0.066382
1,you,149.0,0.103043,3081151.0,0.064643,0.038400
7,all,61.0,0.042185,1069419.0,0.022436,0.019749
9,can,56.0,0.038728,1033493.0,0.021683,0.017045
19,chart,25.0,0.017289,11861.0,0.000249,0.017040
20,moon,24.0,0.016598,13959.0,0.000293,0.016305
18,need,26.0,0.017981,351621.0,0.007377,0.010604
22,change,22.0,0.015214,235609.0,0.004943,0.010271
33,personal,15.0,0.010373,67917.0,0.001425,0.008949
36,powerful,13.0,0.008990,33151.0,0.000696,0.008295


But we now have a lot more horoscopes scraped. Plus we can get as many as we like from The Sun, using the backdated ones. So we can put together a bigger sample, and work towards 5000 words, which seems like the smallest sample we should be using (probably could apply some of the statistical tests I learned - for comparing two populations and finding out if something is significantly higher in one than the other).